In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import scipy.stats as stats
import math
from decimal import *
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import sobol
import nbimporter
from PINN_Solver_Classes import *
from PDE_Classes import *
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
# from keras import callbacks
# import time
# import itertools as product

# Set data type
DTYPE = 'float64'
tf.keras.backend.set_floatx(DTYPE)

In [26]:
N_b = 2**8 # number of boundary and interface points
N_r = 2**10 # number of internal collocation points

# Declare PDE parameters
nu = 0.000001
beta = 1

if nu <= 2e-3:
    mu = np.log(( (nu/beta)**(nu/beta) )*(np.exp(1)))
else:
    mu = (nu/beta)*np.log( ( nu/beta )*( np.exp(beta/nu) - 1 ) )

# Declare bounds of each subdomain
sub = (
      [0, 0.7],
      [0.6, 1]
      )

# Initialize list of points which lie on the boundary
BC = [0, 1]

# Lower bounds as TF scalars 
x0_om = tf.constant([x[0] for x in sub], dtype=DTYPE)
# Upper bounds as TF scalars 
x1_om = tf.constant([x[1] for x in sub], dtype=DTYPE)

# Generate boundary points for each subdomain boundary
# zipped booleans indicate whether a point is a model or interface boundary
X_b = tuple( zip( [tf.constant(np.repeat([[i]], N_b, axis=0), dtype=DTYPE) for i in sub[0]], [k in BC for k in sub[0]] ) ) 


# Set random seed for reproducible results
tf.random.set_seed(0)

# Generate uniform internal points for the whole domain
points = sobol.sample(dimension=1, n_points=N_r-2)
xwidth = BC[1] - BC[0]
x_uni = np.vstack( (np.vstack( ([0], xwidth*points + BC[0]) ), [1]) )

# # Generate internal points with gaussian distribution centered at the shock
# sigma = (x1_om[-1] - x0_om[0])/2
# x_gauss = np.array(stats.truncnorm((x0_om[0] - mu) / sigma, (x1_om[-1] - mu) / sigma, loc=mu, scale=sigma).rvs(N_r))

# # Take as the final sample set the union of the two sets defined above
# x = np.union1d(x_uni, x_gauss)

# Store internal points as tensor object
X = tf.constant(x_uni, shape=(len(x_uni), 1), dtype=DTYPE)

# Split internal points by subdomain and store in tuple
mask = (X >= x0_om[0]) & (X <= x1_om[0])
temp = tf.boolean_mask(X, mask)
X_r = tf.constant(temp, shape=(temp.shape[0],1), dtype=DTYPE)

In [ ]:
# Declare PDE first as a class for the PINN solver to inherit
pde = PDE_1D_Steady_AdvecDiff

# Then declare an instance of the PDE class to use in the Driver
order = 2
pde1 = pde(nu=nu, beta=beta, order=order)

X_om = [tf.constant(np.linspace(s[0], s[1], num=N_r), shape=(N_r, 1), dtype=DTYPE) for s in sub]

xl = sub[-1][0]
xr = sub[-1][1]

x_FD = X_om[1]

print(tf.reshape(x_FD[0], shape=(1,1)))

h = x_FD[1] - x_FD[0]

a = - nu/(h**2)
b = (2*nu)/(h**2)
c = -(nu/(h**2))

if order == 1:
    b += beta / h
    c += -beta / h
    d = 0.0
elif order == 2:
    b += 3 / 2 * beta / h
    c += -2 * beta / h
    d = 1 / 2 * beta / h

A = np.diagflat([b]*(N_r-1)) + np.diagflat([c]*(N_r - 2), -1) + np.diagflat([a]*(N_r - 2), 1) + np.diagflat([d]*(N_r - 3), -2)

f = np.ones((N_r-1, 1))

f[0] += -d*np.random.rand(1) - c*np.random.rand(1)
f[1] += -d*np.random.rand(1)
#f[-1] += -a*pde1.f(xr) 

u_FD = np.linalg.solve( A, f )

u_FD = np.hstack((u_FD.flatten(), pde1.f(xr)))

u_int = np.interp(x1_om[0], x_FD[:,0], u_FD)


# x_true = np.linspace(0, 1, num=5001)
# u_true = pde1.f(x_true)

# fig, ax = plt.subplots(1,1, figsize=(10,4), dpi=600)

# ax.plot(x_true, u_true, 'b-', x_FD, u_FD, 'r-')
# ax.set_xlabel('x', fontsize=18)
# ax.set_ylabel('u(x)', fontsize=18)

In [28]:
# Build a neural network to handle each domain and intialize data frame for loss
column_list = []
model_r = PINN_Architecture(x0_om[0], x1_om[0])
model_r.build(input_shape=(None, X_r.shape[1]))

for i in range(len(sub)):
    column_list += ["model {:d} Phi_r".format(i+1), 
                    "model {:d} Phi_i".format(i+1), "model {:d} Loss".format(i+1)]

loss_frame = pd.DataFrame(columns=column_list)

# Define learning rate schedule and choose optimizer (Adam)
lr = 1e-3 #tf.keras.optimizers.schedules.PiecewiseConstantDecay([2**6, 2**7],[1e-2,5e-3,1e-3])

# Declare hyperparameters
alpha = 0.25
numEpochs = 2**9
snap = 2**6

In [ ]:
# Initialize Schwarz tolerance, iteration variables, and random initial guess for u
d_tol = 1e-3
schwarz_conv = 1
iterCount = 0
u_i_minus1 = tf.constant(np.random.rand(N_r,1), shape=(N_r, 1), dtype=DTYPE)

# Initialize variables for plotting Schwarz results
x = tf.constant(np.linspace(0, 1, num=N_r), shape=(N_r, 1), dtype=DTYPE)
u_true = pde1.f(x)
fig = plt.figure()
subplot_rows=0
subplot_col=2
fig_store = ()
Schwarz_err = ()

strong = 0
# Main Schwarz loop
while schwarz_conv > d_tol:
    
    # Update plot format with increasing iterations
    if iterCount%subplot_col == 0:
        subplot_rows+=1
        gs = GridSpec(subplot_rows,subplot_col)
        for i, ax in enumerate(fig.axes):
            ax.set_position(gs[i].get_position(fig))
            ax.set_subplotspec(gs[i])
        fig.tight_layout(pad=1.0)
        fig.set_size_inches(3+subplot_rows*3, subplot_rows*3)
    
    ax = fig.add_subplot(subplot_rows,subplot_col,(iterCount%10 + 1))
    ax.set_xlabel('x', fontsize=16)
    ax.set_ylabel('u(x)', fontsize=16)
    
    iterCount += 1
    ax.set_title('Schwarz iteration {:d}'.format(iterCount), fontsize=16)

    ax.plot(x, u_true, 'k--')
    
    # intialize a new figure every 10 iterations to save new pages of results
    if iterCount%10 == 0:
        fig_store += (fig,)
        fig = plt.figure()
        fig.tight_layout(pad=1.0)
        subplot_rows=0
        subplot_col=2 
    
    # initialize tuples to contain loss for each PINN and current approximation, u, for each PINN
    loss_list = ()
    u_i_om = ()

    # Set batch size equal to number of points
    batch_size = len(X_r)
    
    u_int = np.interp(x1_om[0], x_FD[:,0], u_FD)

    # Initialize solver
    p = FD_PINN_Schwarz_Steady(model_r, u_int, X_r, X_b, alpha, pde, strong, snap=snap, nu=nu, beta=beta)

    # Solve PINN
    p.solve(tf.keras.optimizers.Adam(learning_rate=lr), batch_size, numEpochs)

    # Output loss results for each model
    print('Model {:d}: '.format(1))
    print('\t'+'Residual loss = {:10.8e}'.format(p.phi_r))
    print('\t'+'Interface loss = {:10.8e}'.format(p.phi_i))
    if not strong:
        print('\t'+'Boundary loss = {:10.8e}'.format(p.phi_b))
    if snap:
        print('\t'+'Snapshot loss = {:10.8e}'.format(p.phi_s))
    print('\t'+'Total loss = {:10.8e}'.format(p.loss))
    loss_list += (p.phi_r, p.phi_i, p.loss)
    
    f_NN = np.ones((A.shape[0], 1))
    if strong:
        #f_NN[0] += -d*p.BC_enforce(x_FD[0])*model_r(tf.reshape(x_FD[0], shape=(1,1))) - c*pde1.f(x_FD[1])
        f_NN[0] += ( -d*p.BC_enforce(x_FD[0])*model_r(tf.reshape(x_FD[0], shape=(1,1))) 
                    - c*p.BC_enforce(x_FD[1])*model_r(tf.reshape(x_FD[1], shape=(1,1))) )
            
        #f_NN[1] += -d*pde1.f(x_FD[1]) 
        f_NN[1] += -d*p.BC_enforce(x_FD[1])*model_r(tf.reshape(x_FD[1], shape=(1,1))) 
    else:    
        #f_NN[0] += -d*model_r(tf.reshape(x_FD[0], shape=(1,1))) - c*pde1.f(x_FD[1])
        f_NN[0] += -d*model_r(tf.reshape(x_FD[0], shape=(1,1))) - c*model_r(tf.reshape(x_FD[1], shape=(1,1)))
        
        #f_NN[1] += -d*pde1.f(x_FD[1]) 
        f_NN[1] += -d*model_r(tf.reshape(x_FD[1], shape=(1,1)))
    
    u_FD = np.linalg.solve( A, f_NN )
    u_FD = np.hstack((u_FD.flatten(), pde1.f(xr)))
    u_int = np.interp(x1_om[0], x_FD[:,0], u_FD)

    # Save current PINN approximations and add them to plot for current Schwarz iteration
    if strong:
        u_i_om = ( p.BC_enforce(X_om[0])*model_r(X_om[0]), tf.reshape(u_FD, shape = (len(u_FD), 1)) )
    else:
        u_i_om = ( model_r(X_om[0]), tf.reshape(u_FD, shape = (len(u_FD), 1)) )

    ax.plot(X_om[0], u_i_om[0], x_FD, u_i_om[1])
  
    # Combine model approximations for each domain
    u_i = sum(u_i_om)/len(u_i_om)

    # Calculate the normalized difference between u for the current iteration and u for the previous iteration
    schwarz_conv = tf.math.reduce_euclidean_norm(u_i - u_i_minus1)/tf.math.reduce_euclidean_norm(u_i)
    
    # Update u for the previous iteration
    u_i_minus1 = u_i
    
    # Output current Schwarz error 
    print('\nSchwarz iteration {:d}: error = {:10.8e}'.format(iterCount, schwarz_conv), "\n")
    
    # Record loss and schwarz error history
    #loss_frame.loc[iterCount-1] = loss_list
    Schwarz_err += (schwarz_conv,)


In [32]:
gs = GridSpec(subplot_rows,subplot_col)
for i, ax in enumerate(fig.axes):
    ax.set_position(gs[i].get_position(fig))
    ax.set_subplotspec(gs[i])
fig.tight_layout(pad=1.0)
fig.set_size_inches(3+subplot_rows*3, subplot_rows*3)

# Save all figures as PDFs
fig_store += (fig,)
for i,figure in enumerate(fig_store):
    figure.savefig("C:/Users/wdsnyde/code/fhnm-ldrd/Docs/Schwarz-PINNs/SchwarzIter_AdvecDiff_FD-PINN_Weak_nu_{:1.0e}_beta_{:1.0e}_pg{:d}.pdf".format(nu,beta,i))

In [6]:
# Generate internal points with gaussian distribution about the shock
# mu = (nu/beta)*np.log( ( nu/beta )*( np.exp(beta/nu) - 1 ) )
# sigma = (x1_om[-1] - x0_om[0])/1.5
# x = stats.truncnorm((x0_om[0] - mu) / sigma, (x1_om[-1] - mu) / sigma, loc=mu, scale=sigma).rvs(N_r)

# def get_curvature(x):
#     u_x = (1/beta) - ( (beta/nu)*np.exp(beta*x/nu) )/( beta*(np.exp(beta/nu) - 1) )
#     u_xx = -((beta/nu)**2)*np.exp(beta*x/nu)/( beta*(np.exp(beta/nu) - 1) )

#     return np.abs(u_xx)/( (1 + u_x**2)**(3/2) )
  
# sample = 0
# max_length = 1/(2**15)
# max_curve_grad = 0.05
# x = np.array([sample])

# while sample < 1:
#     sample_next = sample + max_length
    
#     k_diff = np.abs(get_curvature(sample) - get_curvature(sample_next))
    
#     if not k_diff > max_curve_grad:
#         x = np.concatenate(( x, np.array([sample_next]) ), axis=0)
#         sample = sample_next
#         continue
    
#     sub_div = math.ceil(k_diff/max_curve_grad)
    
#     x = np.concatenate(( x, np.linspace(sample, sample_next, num=sub_div)[1:] ), axis=0)
    
#     sample = sample_next
##########################################################################################